# Fuzzy upravljanje poljoprivrednom proizvodnjom u plasteniku

<center><img src="Dinko.png" alt="drawing" width="50%"/></center>

### KRATAK OPIS PROBLEMA 
<p>
 Kao što nam je poznato u plastenicima i staklenicima se vrlo brzo mijenjaju klimatski uvjeti
 i zbog toga nam je potrebna automatska regulacija ventilacije (temperature) kako bi se potpuno iskoristila funkcionalnost objekata.

Zbog previsoke temperature u objektu dolazi do izduživanja internodija na biljkama dok preniska 
temperatura zaustavlja njezin rast. Kod ručnog otvaranja ventilacije nikad ne možemo pravovremeno 
odrediti kada je potrebno otvaranje ili zatvaranje ventilacije, a već mala zakašnjenja mogu dovesti do
 ekstremno visokih temperatura što je veliki šok za biljke.

Sa automatikom se ventilacija vrši postepenim otvaranjem prozora čime se onemogućavaju nagle 
promijene temperature koje uzrokuju šokiranje biljaka. Kod određene brzine i smjera vjetra automatika 
po potrebi pritvara ili zatvara ventilacijske otvore kako bi se spriječilo oštećenje objekta kod nevrjemena.<sup>[1](#izvori)</sup>
</p>

### DEFINICIJA ULAZNIH I IZLAZNIH VARIJABLI

Projekat bi bio realiziran na platformi Raspberry pi pico w, zbog svoje podrške za Python programiranje i zato što ima ugrađen WiFi čip na sebi koji ćemo iskoristiti u projektu. U nastavku su definisane ulazne i izlazne varijable koje ćemo koristiti za mikrokontroler:

#### Ulazne varijable

Za ulazne varijable smo odabrali sljedećih 5: temperatura i vlažnost zraka, brzina vjetra, AQ indeks i da li pada kiša. Temperatura zraka je po prirodi kontinualna varijabla, koju ćemo mjeriti pomoću `DHT 22` senzora, koji može da mjeri temperaturu u rasponu [-40°C, 80°C]. ~~Kategorički ćemo temperaturu podjeliti na `hladno`, `toplo`, `vruće`~~. Vlažnost zraka je također po prirodi kontinualna varijabla, koju ćemo također mjeriti pomoću `DHT 22` senzora, koji može mjeriti vlažnost zraka u rasponu [0%, 100%].<sup>[2](#izvori)</sup> ~~Kategorički ćemo vlažnost podjeliti na `vlažno`, `suho`~~. Brzinu vjetra je kontinualna varijabla po prirodi i nju ćemo mjeriti pomoću `adafruit anemometara`. Ovaj a anemometar može mjeriti brzinu vjetra u rasponu [0m/s, 32.4m/s]. ~~Kategorički ćemo vjetar podjeliti na `nema`, `slab`, `umjeren` i `jak`~~. `AQ indeks` je kontinualna varijabla za čije mjerenje ćemo koristiti API sa stranice https://aqicn.org. Međutim iz sigurnosnih razloga, da bi zaštitili API ključ koji smo dobili za pristup njihovom API-ju, napravili smo novi servis (na domeni https://dajposlic.pythonanywhere.com/) kroz koji ćemo pristupati njihovom API-ju kroz ovaj zadatak. raspon AQ indeksa je [0, +∞]. ~~AQ indeks je kategorisan na sljedeći način: `dobro`, `umjereno`, `blago nezdravo`, `nezdravo`, `pravo nezdravo`, `opasno`<sup>[3](#izvori)</sup>~~. Varijabla `da li pada kiša` je diskretna varijabla, koju ćemo mjeriti pomoću `senzora za kišu`, koji daje vrijednosti  {ne, da}. 

#### Izlazne varijable
Izlazna varijabla će nam biti tipa ~~__#Bool mikrokontroleru kao signal da treba otvoriti prozore#Realan broj(Double) koji predstavlja koliko će mikrokontroler otvoriti prozore#__~~


### DEFINICIJA ULAZNIH I IZLAZNIH VARIJABLI

#### Ulazne varijable
~~d
Ulazne varijable koje ćemo koristiti za naš fuzzy kontroler su: 

1.    Temperatura zraka
1.    Brzina vjetra
1.    Smjer vjetra
1.    Da li pada kiša
1.    AQI indeks zraka

Senzori koji se koriste za ventilaciju:

1.    Unutarnji temperaturni senzor
1.    Senzor za mjerenje brzine vjetra
1.    Senzor za kišu
1.    Senzor za smjer vjetra

Vanjski API za AQ indeks zraka koji ćemo koristiti je sa stranice https://aqicn.org, ali koristimo svoj REST endpoint:
1.    https://dajposlic.pythonanywhere.com/AQI

#### Izlazne varijable

Izlazna varijabla će nam biti tipa ~~__#Bool mikrokontroleru kao signal da treba otvoriti prozore#Realan broj(Double) koji predstavlja koliko će mikrokontroler otvoriti prozore#__~~
d~~


### DEFINISANJE FUZZY SKUPOVA

### IMPLEMENTACIJA FUZZY SISTEMA

### ZAKLJUČAK I KRATKA REFLEKSIJA

### IZVORI

1. https://www.winter.hr/automatika.htm
1. https://cdn.sparkfun.com/assets/f/7/d/9/c/DHT22.pdf
1. https://waqi.info/#/c/3.396/8.206/2.2z



### ULOGE ČLANOVA